In [1]:
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")

In [2]:
import torch
import os
print("Cuda is available : ", torch.cuda.is_available())
print("Cuda num : ", torch.cuda.device_count())  # gpu数量
print("Current cuda index : ", torch.cuda.current_device())
print("Current cuda name : ", torch.cuda.get_device_name(0))


Cuda is available :  True
Cuda num :  1
Current cuda index :  0
Current cuda name :  NVIDIA GeForce RTX 3090


In [3]:
from libs import *
dataPath = "../StockPriceForecast/dataSet/data/marketData/data.nc"
data_len = DataLength(dataPath)
print("All days : ", data_len)
input_days = 360
predict_days = 30
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[0, data_len - input_days - test_predict_len],
                        encoderDecoderParameter="close",
                        targetDataParameter="close")
train, target = train_dataSet[0]
print("Train days : ", train_dataSet.data.shape)
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len])
print("Validation days ; ", validation_dataSet.data.shape)
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len])
print("Test days : ", test_dataSet.data.shape)

train_dataLoader = DataLoader(train_dataSet)
validation_dataLoader = DataLoader(validation_dataSet)
test_dataLoader = DataLoader(test_dataSet)

All days :  2994
target_data :  torch.Size([30, 1440, 1])
Train days :  (2274, 1440, 10)
Validation days ;  (720, 1440, 10)
Test days :  (720, 1440, 10)


In [4]:
from os.path import join
models_store_path = "../StockPriceForecast/Models/"
days_num = 100
loss_function_name = "L1"
lr = 0.00005
num_epochs = 1
model_name = f"spf_{loss_function_name}_{lr}_{num_epochs}_{days_num}.pth"
model_store_path = join(models_store_path, model_name)
train_dataSet = train_dataSet.isel(0, 390 + days_num -1)
train_dataLoader = DataLoader(train_dataSet)
print(model_store_path)

../StockPriceForecast/Models/spf_L1_5e-05_1_100.pth


In [5]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
model = Seq2SeqAttention(input_size=14400,
                         hidden_size=4096,
                         output_size=1440,
                         num_layers=4,
                         dropout=0.1)
model = train_Seq2SeqAttention(model=model,
                               device=device,
                               dataLoader=train_dataLoader,
                               loss_function_name=loss_function_name,
                               lr=lr,
                               num_epochs=num_epochs)
torch.save(model, model_store_path)

epoch 0 training:   0%|          | 0/100 [00:00<?, ?it/s]


target_data :  torch.Size([30, 1440, 10])
enc_input[-1].unsqueeze(dim=0) :  torch.Size([1, 1, 14400])
dec_input[:-1]] :  torch.Size([29, 1440, 10])


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 1 but got size 1440 for tensor number 1 in the list.

In [ ]:
validation_dataLoader.isel(0, 1)
predicts, targets, attention_weights = eval_Seq2SeqAttention(model=model,
                                                    dataLoader=validation_dataLoader,
                                                    steps=30,
                                                    device=device)